# import

In [1]:
import pandas as pd
import numpy as np
import dplython
import Levenshtein
import pyodbc
import time
from progressbar import *
from sqlalchemy import create_engine
import urllib

In [2]:
cnxn =pyodbc.connect('DRIVER={SQL Server};SERVER=172.16.5.37;DATABASE=國際風險資訊串流輔助系統;UID=fdanrt;PWD=VGdS8lznYmWEh4rt')
params = urllib.parse.quote_plus('DRIVER={SQL Server};SERVER=172.16.5.37;DATABASE=國際風險資訊串流輔助系統;UID=fdanrt;PWD=VGdS8lznYmWEh4rt')
conn = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params).connect()

In [ ]:
data = pd.read_sql("select * from 國際警訊產品名稱_邊境 IFI  cross join 國際警訊產品名稱 IFN where IFI.狀態='N'or IFN.狀態='N'",cnxn)

In [7]:
pd.read_sql("select * from 國際警訊產品名稱_邊境",cnxn)

,IFI_PK,貨物名稱英文品名_c,date,狀態


# prototype

In [ ]:
IFI=data.iloc[:,0]
A=data.iloc[:,1]
B=data.iloc[:,5]

In [ ]:
tStart = time.time()#計時開始
 
#模擬要測量的function
progress = ProgressBar()
score=[[Levenshtein.jaro(A[i],B[i])] for i in progress(range(len(data)))]
tEnd = time.time()#計時結束
#列印結果
print ("It cost %f sec" % (tEnd - tStart))#會自動做近位

score=pd.DataFrame(score,columns=['score'])

In [ ]:
result=pd.concat([IFI,A,B,score],axis=1)
result.columns=['IFI_PK','IFI_貨物名稱英文品名_c','IFN_產品名稱_c','score']
output=result[result['score']>0.8]
print(output)
output.to_sql(name='模糊比對_產品名稱比對結果_2', con=conn, if_exists='append', index=False)

In [ ]:
cursor = cnxn.cursor()
cursor.execute('''
                UPDATE 國際警訊產品名稱_邊境
                SET 狀態='Y'
                WHERE 狀態='N'
                UPDATE 國際警訊產品名稱
                SET 狀態='Y'
                WHERE 狀態='N'
                ''')
cnxn.commit()

## rename column

In [ ]:
cursor = cnxn.cursor()
cursor.execute("EXEC sp_rename '模糊比對_產品名稱比對結果.IFI_貨物名稱英文品名_c', 'IFI_產品名稱_c', 'COLUMN'")
cnxn.commit()

In [ ]:
cursor = cnxn.cursor()
cursor.execute("EXEC sp_rename '模糊比對_品牌比對結果.IFI_商標_c', 'IFI_品牌_c', 'COLUMN'")
cnxn.commit()

# function

## join兩個表選N or N

In [9]:
def compare(x,y=1):
    if y==1: y=x
    else: y='商標'
    print('---------%s模糊比對----------'%x)
    data = pd.read_sql("select * from 國際警訊%s_邊境 IFI  cross join 國際警訊%s IFN where IFI.狀態='N'or IFN.狀態='N'"%(y,x),cnxn)
    
    IFI=data.iloc[:,0]
    A=data.iloc[:,1]
    B=data.iloc[:,5]


    progress = ProgressBar()
    score=[[Levenshtein.jaro(A[i],B[i])] for i in progress(range(len(data)))]

    score=pd.DataFrame(score,columns=['score'])


    result=pd.concat([IFI,A,B,score],axis=1)
    result.columns=['IFI_PK','IFI_%s_c'%x,'IFN_%s_c'%x,'score']
    output=result[result['score']>0.8]
    print(output)

    output.to_sql(name='模糊比對_%s比對結果'%x, con=conn, if_exists='append', index=False)


    cursor = cnxn.cursor()
    cursor.execute('''
                    UPDATE 國際警訊%s_邊境
                    SET 狀態='Y'
                    WHERE 狀態='N'
                    UPDATE 國際警訊%s
                    SET 狀態='Y'
                    WHERE 狀態='N'
                    '''%(y,x))
    cnxn.commit()

## FOR迴圈方式做兩次，因為join資料量太大(append)

In [ ]:
def compare(x):
    print('---------%s模糊比對----------'%x)
    
    data = pd.read_sql("select * from 國際警訊%s_邊境 IFI where 狀態='N'"%x,cnxn)
    data2 = pd.read_sql("select * from 國際警訊%s"%x,cnxn)

    if data.empty!=True:
        IFI=data.iloc[:,0]
        A=data.iloc[:,1]
        B=data2.iloc[:,1]

        progress = ProgressBar()
        for i in progress(range(len(A))):
            score=[[Levenshtein.jaro(A[i],B[j])] for j in range(len(B))]
            score=pd.DataFrame(score,columns=['score'])

            IFIA=pd.DataFrame([[IFI[i],A[i]] for j in range(len(B))])

            result=pd.concat([IFIA,B,score],axis=1)
            if i==0:output=result[result['score']>0.8]
            else:output=output.append(result[result['score']>0.8])

        output.columns=['IFI_PK','IFI_%s_c'%x,'IFN_%s_c'%x,'score']

        print(output)

        output.to_sql(name='模糊比對_%s比對結果'%x, con=conn, if_exists='append', index=False)
##    
    data = pd.read_sql("select * from 國際警訊%s_邊境 IFI"%x,cnxn)
    data2 = pd.read_sql("select * from 國際警訊%s IFI where 狀態='N'"%x,cnxn)

    if data2.empty!=True:
        IFI=data.iloc[:,0]
        A=data.iloc[:,1]
        B=data2.iloc[:,1]
        progress = ProgressBar()
        for i in progress(range(len(A))):
            score=[[Levenshtein.jaro(A[i],B[j])] for j in range(len(B))]
            score=pd.DataFrame(score,columns=['score'])

            IFIA=pd.DataFrame([[IFI[i],A[i]] for j in range(len(B))])

            result=pd.concat([IFIA,B,score],axis=1)
            if i==0:output=result[result['score']>0.8]
            else:output=output.append(result[result['score']>0.8])

        output.columns=['IFI_PK','IFI_%s_c'%x,'IFN_%s_c'%x,'score']

        print(output)

        output.to_sql(name='模糊比對_%s比對結果'%x, con=conn, if_exists='append', index=False)

    cursor = cnxn.cursor()
    cursor.execute('''
                    UPDATE 國際警訊%s_邊境
                    SET 狀態='Y'
                    WHERE 狀態='N'
                    UPDATE 國際警訊%s
                    SET 狀態='Y'
                    WHERE 狀態='N'
                    '''%(x,x))
    cnxn.commit()

In [4]:
compare('產品名稱')
compare('製造廠')
compare('品牌')

100% |########################################################################|


         IFI_PK          IFI_產品名稱_c               IFN_產品名稱_c     score
1672763  527007  VEGETABLESFOILBAGS  VEGETABLEMEDLEY28OZBAGS  0.806280
1918028  529182    VEGETABLENOODLES  VEGETABLEMEDLEY28OZBAGS  0.804089


# 比對全部(用replace)

In [ ]:
x='產品名稱'
data = pd.read_sql("select * from 國際警訊%s_邊境 IFI"%x,cnxn)

IFI=data.iloc[:,0]
A=data.iloc[:,1]

data2 = pd.read_sql("select * from 國際警訊%s IFI"%x,cnxn)
B=data2.iloc[:,1]

R=lambda x:pd.concat([x[x['score']>0.6], x[x['score']==x['score'].max()]]).drop_duplicates()

progress = ProgressBar()
for i in progress(range(len(A))):
    score=[[Levenshtein.jaro(A[i],B[j])] for j in range(len(B))]
    score=pd.DataFrame(score,columns=['score'])

    IFIA=pd.DataFrame([[IFI[i],A[i]] for j in range(len(B))])
    
    result=pd.concat([IFIA,B,score],axis=1)
    if i==0:output=R(result)
    else:output=output.append(R(result))
    
output.columns=['IFI_PK','IFI_%s_c'%x,'IFN_%s_c'%x,'score']

print(output)

output.to_sql(name='模糊比對_%s比對結果'%x, con=conn, if_exists='replace', index=False)

cursor = cnxn.cursor()
cursor.execute('''
                UPDATE 國際警訊%s_邊境
                SET 狀態='Y'
                WHERE 狀態='N'
                UPDATE 國際警訊%s
                SET 狀態='Y'
                WHERE 狀態='N'
                '''%(x,x))
cnxn.commit()

# history_1204

In [8]:
# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import dplython
import Levenshtein
import pyodbc
#compar = pd.read_csv('C:/Users/tony/Desktop/1003_模糊比對/國際警訊製造廠_utf8.csv',encoding='UTF-8')
#data = pd.read_csv('C:/Users/tony/Desktop/1003_模糊比對/國際警訊製造廠_邊境_utf8.csv',encoding='UTF-8')

In [3]:
compar = pd.read_excel (r'模糊比對資料來源.xlsx', sheet_name=r'國際警訊產品名稱',encoding='UTF-8')
data = pd.read_excel (r'模糊比對資料來源.xlsx', sheet_name=r'國際警訊產品名稱_邊境',encoding='UTF-8')

In [4]:
# cnxn =pyodbc.connect('DRIVER={SQL Server};SERVER=172.16.5.37;DATABASE=國際風險資訊串流輔助系統;UID=fdanrt;PWD=VGdS8lznYmWEh4rt')
# cursor = cnxn.cursor()
# sql="select * from 國際警訊產品名稱_邊境"
# data = pd.read_sql(sql,cnxn)
# cursor.close()
# cursor = cnxn.cursor()
# sql="select * from 國際警訊產品名稱"
# compar = pd.read_sql(sql,cnxn)
# compar  = pd.DataFrame(compar)
# data  = pd.DataFrame(data)
product = np.array(pd.DataFrame(data['貨物名稱英文品名_c']))
n_product = compar['產品名稱_c'].unique()
product_df = data
result=[]

In [5]:
np.array([Levenshtein.jaro(product[i][0],i2) for i in range(10) for i2 in n_product[1:5]])
# np.argmax(a, axis=0)[0]

array([0.        , 0.        , 0.34612512, 0.34369351, 0.49304871,
       0.41904762, 0.45714286, 0.4270179 , 0.52426564, 0.35925926,
       0.51693122, 0.53506698, 0.43124962, 0.37647059, 0.46920922,
       0.43886455, 0.50055539, 0.35163399, 0.48991597, 0.44233069,
       0.45987006, 0.42898551, 0.40046584, 0.43500531, 0.35337819,
       0.40813008, 0.41533827, 0.4695754 , 0.        , 0.        ,
       0.3543514 , 0.35191978, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ])

In [6]:
np.array([Levenshtein.jaro(product[i][0],i2) for i in range(len(product)-1) for i2 in n_product])

C:\Users\6506\AppData\Local\Continuum\anaconda3\lib\site-packages\_pytest\mark\structures.py:378: DeprecationWarning: The usage of `cmp` is deprecated and will be removed on or after 2021-06-01.  Please use `eq` and `order` instead.
  @attr.s(cmp=False, hash=False)


TypeError: jaro expected two Strings or two Unicodes

## 兩兩比對

In [51]:
cnxn =pyodbc.connect('DRIVER={SQL Server};SERVER=172.16.5.37;DATABASE=國際風險資訊串流輔助系統;UID=fdanrt;PWD=VGdS8lznYmWEh4rt')
cursor = cnxn.cursor()
sql="select * from 模糊比對_產品名稱比對結果_2 where score>0.9"
data = pd.read_sql(sql,cnxn)
cursor.close()
data

,IFI_PK,IFI_貨物名稱英文品名_c,IFN_產品名稱_c,score
0,251432.0,FRIEDFISHCAKE,FRIEDFISHCAKE,1.000000
1,340538.0,HOTRICECAKESOUP,RICECAKESOUP,0.933333
2,341153.0,HSVANILLAICECREAM,VANILLAICECREAM,0.960784
3,380700.0,MANGO,MANGO,1.000000
4,380987.0,MANGOICECREAM,MANGOICECREAM,1.000000
...,...,...,...,...
357,230495.0,FISHCRACKERS,FISHCAKES,0.916667
358,230653.0,FISHCAKE,FISHCAKES,0.962963
359,230740.0,FISHTOFU,FISHTOFU,1.000000
360,231663.0,FISHMILT,FISHMINT,0.916667


In [38]:
a=[[Levenshtein.jaro(data.iloc[i,1],data.iloc[i,2])] for i in range(len(data))]
a=pd.DataFrame(a,columns=['score'])

In [45]:
result=pd.concat([data.iloc[:,0:3],a],axis=1)

## NNN

In [ ]:
# %load nnn.py
#!/usr/bin/env python

# In[1]:


# -*- coding: utf-8 -*-
import pandas as pd
import numpy as np
import dplython
import Levenshtein
import pyodbc
#compar = pd.read_csv('C:/Users/tony/Desktop/1003_模糊比對/國際警訊製造廠_utf8.csv',encoding='UTF-8')
#data = pd.read_csv('C:/Users/tony/Desktop/1003_模糊比對/國際警訊製造廠_邊境_utf8.csv',encoding='UTF-8')


# In[3]:


cnxn =pyodbc.connect('DRIVER={SQL Server};SERVER=172.16.5.37;DATABASE=國際風險資訊串流輔助系統;UID=fdanrt;PWD=VGdS8lznYmWEh4rt')
cursor = cnxn.cursor()
sql="select * from 國際警訊產品名稱_邊境"
data = pd.read_sql(sql,cnxn)
cursor.close()
cursor = cnxn.cursor()
sql="select * from 國際警訊產品名稱"
compar = pd.read_sql(sql,cnxn)
compar  = pd.DataFrame(compar)
data  = pd.DataFrame(data)
product = np.array(pd.DataFrame(data['貨物名稱英文品名_c']))
n_product = compar['產品名稱_c'].unique()
product_df = pd.DataFrame(data)
result=[]


# In[4]:


l_product = len(product)


# In[5]:


for i in range(l_product-1):
    a=np.array([[Levenshtein.jaro(product[i][0],i2)] for i2 in n_product])
    index= np.argmax(a, axis=0)[0]
    score = a[index][0]
    product_df_temp = product_df[product_df['貨物名稱英文品名_c']==product[i][0]]
    temp = [product_df_temp['IFI_PK'][i],product_df_temp['貨物名稱英文品名_c'][i],n_product[index],score]
    result = pd.DataFrame(result).append(pd.Series(temp), ignore_index=True)
    
result = pd.DataFrame(result)
result.columns=['IFI_PK','IFI_貨物名稱英文品名_c','IFN_產品名稱_c','score']
result.to_csv('C:/Users/fdanrt/Documents/產品名稱_1203.csv', sep=',', encoding='utf-8')





# In[ ]:


from sqlalchemy import create_engine
import urllib
params = urllib.parse.quote_plus('DRIVER={SQL Server};SERVER=172.16.5.37;DATABASE=國際風險資訊串流輔助系統;UID=fdanrt;PWD=VGdS8lznYmWEh4rt')
cnxn = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
con = cnxn.connect()
state=1
if state==0:
    result.to_sql(name='模糊比對_產品名稱比對結果', con=con, if_exists='append', index=False)
else:
    result.to_sql(name='模糊比對_產品名稱比對結果', con=con, if_exists='replace', index=False)
    
con.close()
cnxn.close()


# In[16]:


cnxn =pyodbc.connect('DRIVER={SQL Server};SERVER=172.16.5.37;DATABASE=國際風險資訊串流輔助系統;UID=fdanrt;PWD=VGdS8lznYmWEh4rt')
cursor = cnxn.cursor()
sql="select * from 國際警訊製造廠_邊境"
data = pd.read_sql(sql,cnxn)
cursor = cnxn.cursor()
sql="select * from 國際警訊製造廠"
compar = pd.read_sql(sql,cnxn)
cursor.close()
compar  = pd.DataFrame(compar)
data  = pd.DataFrame(data)
product = np.array(pd.DataFrame(data['製造廠_c']))
n_product = compar['製造廠_c'].unique()
product_df = pd.DataFrame(data)
result=[]




# In[17]:


for i in range(len(product)-1):
    a=np.array([[Levenshtein.jaro_winkler(product[i][0],i2)] for i2 in n_product])
    index= np.argmax(a, axis=0)[0]
    score = a[index][0]
    product_df_temp = product_df[product_df['製造廠_c']==product[i][0]]
    temp = [product_df_temp['IFI_PK'][i],product_df_temp['製造廠_c'][i],n_product[index],score]
    result = pd.DataFrame(result).append(pd.Series(temp), ignore_index=True)
    


# In[18]:


result = pd.DataFrame(result)
result.columns=['IFI_PK','IFI_製造廠_c','IFN_製造廠_c','score']
result.to_csv('C:/Users/fdanrt/Documents/製造廠商名.csv', sep=',', encoding='utf-8')


# In[19]:


from sqlalchemy import create_engine
import urllib
params = urllib.parse.quote_plus('DRIVER={SQL Server};SERVER=172.16.5.37;DATABASE=國際風險資訊串流輔助系統;UID=fdanrt;PWD=VGdS8lznYmWEh4rt')
cnxn = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
con = cnxn.connect()
state=1
if state==0:
    result.to_sql(name='模糊比對_製造廠比對結果', con=con, if_exists='append', index=False)
else:
    result.to_sql(name='模糊比對_製造廠比對結果', con=con, if_exists='replace', index=False)
    
con.close()
cnxn.close()


# In[6]:


cnxn =pyodbc.connect('DRIVER={SQL Server};SERVER=172.16.5.37;DATABASE=國際風險資訊串流輔助系統;UID=fdanrt;PWD=VGdS8lznYmWEh4rt')
cursor = cnxn.cursor()
sql="select * from 國際警訊品牌_邊境"
data = pd.read_sql(sql,cnxn)
cursor.close()
cursor = cnxn.cursor()
sql="select * from 國際警訊品牌"
compar = pd.read_sql(sql,cnxn)
cursor.close()

compar  = pd.DataFrame(compar)
data  = pd.DataFrame(data)
product = np.array(pd.DataFrame(data['商標_c']))
n_product = compar['品牌_c'].unique()
product_df = pd.DataFrame(data)
result=[]
for i in range(len(product)):
    a=np.array([[Levenshtein.jaro_winkler(product[i][0],i2)] for i2 in n_product])
    index= np.argmax(a, axis=0)[0]
    score = a[index][0]
    product_df_temp = product_df[product_df['商標_c']==product[i][0]]
    temp = [product_df_temp['IFI_PK'][i],product_df_temp['商標_c'][i],n_product[index],score]
    result = pd.DataFrame(result).append(pd.Series(temp), ignore_index=True)
    
result = pd.DataFrame(result)
result.columns=['IFI_PK','IFI_商標_c','IFN_品牌_c','score']
result.to_csv('C:/Users/fdanrt/Documents/品牌.csv', sep=',', encoding='utf-8')

from sqlalchemy import create_engine
import urllib
params = urllib.parse.quote_plus('DRIVER={SQL Server};SERVER=172.16.5.37;DATABASE=國際風險資訊串流輔助系統;UID=fdanrt;PWD=VGdS8lznYmWEh4rt')
cnxn = create_engine("mssql+pyodbc:///?odbc_connect=%s" % params)
con = cnxn.connect()
state=1
if state==0:
    result.to_sql(name='模糊比對_品牌比對結果', con=con, if_exists='append', index=False)
else:
    result.to_sql(name='模糊比對_品牌比對結果', con=con, if_exists='replace', index=False)
    
con.close()
cnxn.close()

